In [1]:
#Load of ionosphere data set
import matplotlib.pyplot as  plt
import pandas as pd
import numpy as np


df=pd.read_csv('prep_data.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head()

,Score,Popularity,Members,genre_Drama,genre_Romance,genre_School,genre_Action,genre_Historical,genre_Martial Arts,genre_Samurai,...,Source_Original,Source_Other,Source_Visual novel,Source_Web manga,Rating_G - All Ages,Rating_PG - Children,Rating_PG-13 - Teens 13 or older,Rating_R - 17+ (violence & profanity),Rating_R+ - Mild Nudity,completed_ratio
0,0.747475,0.047613,0.071032,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,High
1,0.751804,0.062214,0.053783,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,High
2,0.707071,0.064208,0.052411,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,High
3,0.730159,0.016946,0.171986,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,High
4,0.362193,0.244459,0.005741,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Low


In [213]:
# Separation into data and label

#X = df.drop('completed_ratio',axis=1).as_matrix()         # Data
#y = df['completed_ratio'].as_matrix()
X = df.drop('completed_ratio',axis=1)     # Data
y = df['completed_ratio'].values
print(X.shape)
print(y.shape)


(1000, 217)
(1000,)


In [214]:
# Train on data. No parameters to adjust

from sklearn.naive_bayes import GaussianNB       ### Because continuous data
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, train_test_split
import sklearn.neighbors as nb

cv = StratifiedKFold(n_splits=10, random_state=1, shuffle=True) 

gnb = GaussianNB()
cv_scores = cross_val_score(gnb,X=X,y=y,cv=cv)
np.mean(cv_scores)


0.507

In [215]:
from sklearn.model_selection import cross_val_predict  
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

predicted = cross_val_predict(GaussianNB(), X=X, y=y,  cv=cv)  

print(confusion_matrix(y, predicted))
print(accuracy_score(y, predicted))

[[280  26 304   1]
 [  1  18  28   1]
 [ 66  52 207   2]
 [  0   8   4   2]]
0.507


In [216]:
from sklearn.metrics import classification_report

print(classification_report(y, predicted))

              precision    recall  f1-score   support

        High       0.81      0.46      0.58       611
         Low       0.17      0.38      0.24        48
      Medium       0.38      0.63      0.48       327
    Very low       0.33      0.14      0.20        14

    accuracy                           0.51      1000
   macro avg       0.42      0.40      0.37      1000
weighted avg       0.63      0.51      0.53      1000



## Adjusting probability threshold

In [ ]:
# Focus on predict all classes


'''y2 = np.zeros((y.shape))
y2[y=='b']=1
y2[y=='g']=0
'''

y2Class1 = np.zeros((y.shape))
y2Class1[y=='High']=1
y2Class2 = np.zeros((y.shape))
y2Class2[y=='Medium']=1
y2Class3 = np.zeros((y.shape))
y2Class3[y=='Low']=1
y2Class4 = np.zeros((y.shape))
y2Class4[y=='Very low']=1
y2 = [y2Class1, y2Class2, y2Class3, y2Class4]


In [236]:
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings("ignore")

def filterp(th,ProbClass1):
    """ Given a treshold "th" and a set of probabilies of belonging to class 1 "ProbClass1", return predictions """ 
    y=np.zeros(ProbClass1.shape[0])
    for i,v in enumerate(ProbClass1):
        if ProbClass1[i]>th:
            y[i]=1
    return y  

for i in range(4):
  (X_train, X_test,  y_train, y_test) = train_test_split(X, y2[i], test_size=.3, random_state=1)
  clf = GaussianNB()
  lth=[]

  # We do a 10 fold crossvalidation with 10 iterations
  kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
  for train_index, test_index in kf.split(X_train, y_train):
      X_train2, X_test2 = X.iloc[train_index], X.iloc[test_index]
      y_train2, y_test2 = y2[i][train_index], y2[i][test_index]

      # Train with the training data of the iteration 
      clf.fit(X_train2, y_train2)

      # Obtaining probablity predictions for test data of the iterarion
      probs = clf.predict_proba(X_test2)

      # Collect probabilities of belonging to class 1
      ProbClass1 = probs[:,1]
      # Sort probabilities and generate pairs (threshold, f1-for-that-threshold) 
      res = np.array([[th,f1_score(y_test2,filterp(th,ProbClass1),pos_label=1)] for th in np.sort(ProbClass1)])

      # Uncomment the following lines if you want to plot at each iteration how f1-score evolves increasing the threshold 
      #plt.plot(res[:,0],res[:,1])
      #plt.show()

      # Find the threshold that has maximum value of f1-score
      maxF = np.max(res[:,1])
      pl = np.argmax(res[:,1])
      optimal_th = res[pl,0]
      
      # Store the optimal threshold found for the current iteration
      lth.append(optimal_th)

  # Compute the average threshold for all 10 iterations    
  thdef = np.mean(lth)
  print("Selected threshold for class number", i+1 , "in 10-fold cross validation:", thdef)
  print()

  # Train a classifier with the whole training data 
  clf = GaussianNB()
  clf.fit(X_train, y_train)
  # Obtain probabilities for data on test set
  probs = clf.predict_proba(X_test)
  # Generate predictions using probabilities and threshold found on 10 folds cross-validation
  pred = filterp(thdef,probs[:,1])
  # Print results with this prediction vector
  print(classification_report(y_test, pred))

  # Ignore warnings explaining that in some iterations f1 score is 0

  clf = GaussianNB()
  clf.fit(X_train, y_train)
  pred=clf.predict(X_test)
  print(classification_report(y_test, pred))

Selected threshold for class number 1 in 10-fold cross validation: 2.4134582045990436e-16

              precision    recall  f1-score   support

         0.0       0.63      0.55      0.59       111
         1.0       0.75      0.81      0.78       189

    accuracy                           0.71       300
   macro avg       0.69      0.68      0.68       300
weighted avg       0.71      0.71      0.71       300

              precision    recall  f1-score   support

         0.0       0.55      0.78      0.65       111
         1.0       0.83      0.62      0.71       189

    accuracy                           0.68       300
   macro avg       0.69      0.70      0.68       300
weighted avg       0.73      0.68      0.69       300

Selected threshold for class number 2 in 10-fold cross validation: 0.5995169108180375

              precision    recall  f1-score   support

         0.0       0.87      0.46      0.61       205
         1.0       0.42      0.85      0.57        95

    